In [1]:
import numpy as np
import tensorflow as tf
from copy import deepcopy

import os
import re

import datetime

import import_ipynb
from QBert import train_utils, models

from tqdm.notebook import tqdm



importing Jupyter notebook from C:\Users\LGCNS\Documents\GitHub\Q_Bert\QBert\train_utils.ipynb
importing Jupyter notebook from C:\Users\LGCNS\Documents\GitHub\Q_Bert\QBert\models.ipynb


## 해당 파일은 bert.run_pretraining.run_bert_pretrain을 구현하는 것을 목표로 한다.

 - Parameter는 FLAG 형식에서 직접 정의해주는 방식으로 변경하고, Main에서 직접 정의하도록 한다.

## Strategy

In [2]:
tf.config.list_physical_devices() # device 확인

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]

In [3]:
# Strategy 정의

distribution_strategy = 'mirrored' # 'tpu'
num_gpus = 0
all_reduce_alg = None

if distribution_strategy == 'tpu' :
    tpu_address = ""
else :
    tpu_address = None



strategy = train_utils.get_distribution_strategy(
                  distribution_strategy=distribution_strategy,
                  num_gpus=num_gpus,
                  all_reduce_alg=all_reduce_alg,
                  tpu_address=tpu_address)

print ('\n장치의 수: {}'.format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)

장치의 수: 1


## Bert_Config

In [4]:
# sub_model (core_model 필요 Config)

vocab_size = 32000 # 
hidden_size = 768 # Transformer hidden Layers
type_vocab_size = 12 #: The number of types that the 'type_ids' input can take.
num_layers = 12
num_attention_heads = 12
max_seq_length = 256 # 512
dropout_rate = .1
# attention_dropout_rate = .1
inner_dim = 3072
# hidden_act = 'gelu'
initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02)

# Pretrain Model 필요 Config
max_predictions_per_seq = 40


## Input_Files

In [5]:
filenames = ['./Test_Examples.tfrecords']

# Create a description of the features.
feature_description = {
    'input_ids': tf.io.FixedLenFeature([max_seq_length], tf.int64),
    'segment_ids': tf.io.FixedLenFeature([max_seq_length], tf.int64),
    'input_mask': tf.io.FixedLenFeature([max_seq_length], tf.int64),
    'masked_lm_positions': tf.io.FixedLenFeature([max_predictions_per_seq], tf.int64),
    'masked_lm_ids': tf.io.FixedLenFeature([max_predictions_per_seq], tf.int64),
    'masked_lm_weights': tf.io.FixedLenFeature([max_predictions_per_seq], tf.float32),
    'next_sentence_labels': tf.io.FixedLenFeature([1], tf.int64),
}

# keys = feature_description.keys()

In [6]:
def _parse_function(example_proto):
  # Parse the input `tf.train.Example` proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, feature_description)

def _select_data_from_record(record):
    """Filter out features to use for pretraining."""
    x = {
        'input_ids': record['input_ids'],
        'input_mask': record['input_mask'],
        'segment_ids': record['segment_ids'],
        'masked_lm_positions': record['masked_lm_positions'],
        'masked_lm_ids': record['masked_lm_ids'],
        'masked_lm_weights': record['masked_lm_weights'],
    }
    if use_next_sentence_label:
        x['next_sentence_labels'] = record['next_sentence_labels']
    if use_position_id:
        x['position_ids'] = record['position_ids']

    # TODO(hongkuny): Remove the fake labels after migrating bert pretraining.
    if output_fake_labels:
        return (x, record['masked_lm_weights'])
    else:
        return x


In [7]:

BUFFER_SIZE = 100

GLOBAL_BATCH_SIZE = 2
BATCH_SIZE_PER_REPLICA = np.ceil(GLOBAL_BATCH_SIZE // strategy.num_replicas_in_sync)

use_next_sentence_label = True
output_fake_labels = True
use_position_id = False

lr = 1e-3


In [8]:

with strategy.scope():

    train_dataset = tf.data.Dataset.from_tensor_slices(filenames)
    train_dataset = train_dataset.interleave(tf.data.TFRecordDataset
                                             , cycle_length = tf.data.experimental.AUTOTUNE
                                             , num_parallel_calls = tf.data.experimental.AUTOTUNE)
    dataset_inputs = train_dataset.map(_parse_function,
                                       num_parallel_calls=tf.data.experimental.AUTOTUNE) # String to Example
    dataset_inputs_with_labels = dataset_inputs.map(_select_data_from_record,
                                                    num_parallel_calls=tf.data.experimental.AUTOTUNE) # Example to InputData
    ## 본래대로라면 그냥 써도 되지만, 현재 Label이 없는 데이터이기 때문에
    ## max_predictions_per_seq 길이의 허위 정답 (Fake_y)를 삽입하는 mapping function이다.

    dataset = dataset_inputs_with_labels
    dataset = dataset.cache()
    dataset = dataset.repeat()
    dataset = dataset.shuffle(10000, reshuffle_each_iteration = True)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(GLOBAL_BATCH_SIZE)
    
    dist_dataset = strategy.experimental_distribute_dataset(dataset)

## Traning Config

# 훈련

In [9]:
def _replicated_step(inputs):
    """Replicated training step."""

    inputs, labels = inputs
    
    with tf.GradientTape() as tape:
        
        model_outputs = model(inputs, training=True)
        loss = loss_fn(labels, model_outputs)
        # Raw loss is used for reporting in metrics/logs.
        raw_loss = loss
    
        if scale_loss:
            # Scales down the loss for gradients to be invariant from replicas.
            loss = loss / strategy.num_replicas_in_sync

    if isinstance(optimizer, tf.keras.mixed_precision.experimental.LossScaleOptimizer):
        
        with tape:
            scaled_loss = optimizer.get_scaled_loss(loss)
        scaled_grads = tape.gradient(scaled_loss, training_vars)
        grads = optimizer.get_unscaled_gradients(scaled_grads)
        
    else:
        
        grads = tape.gradient(loss, training_vars)
        optimizer.apply_gradients(zip(grads, training_vars))
    
    # For reporting, the metric takes the mean of losses.
    train_loss_metric.update_state(raw_loss)
    
    for metric in train_metrics:
        metric.update_state(labels, model_outputs)

In [10]:
@tf.function
def train_step(iterator, steps):
    
#     def step_fn(inputs):
#         features, labels = inputs

#         with tf.GradientTape() as tape:
            
#             losses = model(features)    
#             losses = losses * (1.0 / GLOBAL_BATCH_SIZE)

#         grads = tape.gradient(losses, model.trainable_variables)
#         optimizer.apply_gradients(list(zip(grads, model.trainable_variables)))
        
#         return losses

    if not isinstance(steps, tf.Tensor):
        raise ValueError('steps should be an Tensor. Python object may cause '
                     'retracing.')

    for _ in tf.range(steps):
        strategy.run(_replicated_step, args=(next(iterator),))

In [11]:
def train_single_step(iterator):
    """Performs a distributed training step.

    Args:
    iterator: the distributed iterator of training datasets.

    Raises:
    ValueError: Any of the arguments or tensor shapes are invalid.
    """
    strategy.run(_replicated_step, args=(next(iterator),))

In [12]:
def steps_to_run(current_step, steps_per_epoch, steps_per_loop):
    """Calculates steps to run on device."""
    if steps_per_loop <= 0:
        raise ValueError('steps_per_loop should be positive integer.')
    if steps_per_loop == 1:
        return steps_per_loop
    remainder_in_epoch = current_step % steps_per_epoch
    if remainder_in_epoch != 0:
        return min(steps_per_epoch - remainder_in_epoch, steps_per_loop)
    else:
        return steps_per_loop

In [13]:
# loss
def loss_fn(fake_y, losses, **unused_args) :
    
    return tf.reduce_mean(losses, axis = -1)
    

## Model Define

In [14]:
train_single_step = tf.function(train_single_step)

In [15]:
steps_per_epoch = 25 #25000
steps_between_evals = 10 #
steps_per_loop = 1
epochs = 50

total_training_steps = steps_per_epoch * epochs
eval_loss_metric = tf.keras.metrics.Mean('traning loss', dtype = tf.float32)
train_loss_metric = tf.keras.metrics.Mean('traning loss', dtype = tf.float32)

scale_loss = True if strategy.num_replicas_in_sync > 1 else False

In [16]:
# optimizer

## Learning Rate Decay

# lr = 1e-4 warmup stage (step <= 10000)
# Decay linearly

init_lr = 1e-4
warmup_steps = int(total_training_steps // 10)
end_lr = 0

lr_schedule = tf.keras.optimizers.schedules.PolynomialDecay(
      initial_learning_rate=init_lr,
      decay_steps=total_training_steps,
      end_learning_rate=end_lr)

lr_schedule = train_utils.WarmUp(
        initial_learning_rate=init_lr,
        decay_schedule_fn=lr_schedule,
        warmup_steps=warmup_steps)

optimizer = train_utils.AdamWeightDecay( 
    learning_rate=lr_schedule,
    weight_decay_rate=0.01,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-6,
    exclude_from_weight_decay=['LayerNorm', 'layer_norm', 'bias'])

In [17]:
with strategy.scope() :

    model, sub_model = models.get_bert_models_fn(vocab_size
                                             , hidden_size
                                             , type_vocab_size
                                             , num_layers
                                             , num_attention_heads
                                             , max_seq_length
                                             , max_predictions_per_seq
                                             , dropout_rate
                                             , inner_dim 
                                             , initializer)
    model.compile(optimizer, loss=loss_fn)

In [18]:
# Data Set
train_iterator = iter(dist_dataset)

In [19]:

train_metrics = model.metrics
train_loss_metric = tf.keras.metrics.Mean('training_loss', dtype=tf.float32)

# callback

## model checkpoint
t = datetime.datetime.now().strftime("%Y%m%d%H%M%S")

checkpoint_dir = './training_checkpoints_{}'.format(t)
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

model_cp = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True),

## Learning Rate Print
callbacks = [model_cp]

# callbacks = tf.keras.callbacks.CallbackList(callbacks) # tf 2.5.0 제공


optimizer = model.optimizer
training_vars = model.trainable_variables

# Callback
for cb in callbacks :
    cb[0].model = model


In [24]:
current_step = optimizer.iterations.numpy()
current_step

10

In [21]:
tf.executing_eagerly()

True

In [23]:
from transformers import BertTokenizerFast
tokenizer_for_load = BertTokenizerFast.from_pretrained('./model/BertTokenizer-3000-32000-vocab.txt'
                                                   , strip_accents=False
                                                   , lowercase=False)

Calling BertTokenizerFast.from_pretrained() with the path to a single file or url is deprecated


In [ ]:
history = {}

for cb in callbacks :
    cb[0].on_train_begin()

while current_step < total_training_steps and not model.stop_training :
    if (current_step % steps_per_epoch == 0) :
#         print("{} EPOCH BEGIN".format(int(current_step // steps_per_epoch) + 1))
        for cb in callbacks :
            cb[0].on_epoch_begin(int(current_step // steps_per_epoch) + 1)

    for cb in callbacks :
#         print("BATCH_BEGIN - {}".format(current_step))
        cb[0].on_batch_begin(current_step)
        
    steps = steps_to_run(current_step, steps_between_evals, steps_per_loop)
    
    if tf.config.list_physical_devices('GPU') :
        for _ in steps :
            train_single_step(train_iterator)
    else :
        train_step(train_iterator, tf.convert_to_tensor(steps, dtype = tf.int32))
        
        # GPU, TPU 아키텍쳐, Iteration HOST ,
        # TPU N장 data 각각 들어감.
        # GPU는 나눠준다.

    train_loss = train_loss_metric.result().numpy().astype(float)
    current_step += steps

    logs = {'training_loss' : train_loss}

    for t_metric in train_metrics :
        logs[t_metric.name] = t_metric.result().numpy()

    for key in logs.keys() :
        if key in history.keys() :
            history[key].append(logs[key])
        else :
            history[key] = [logs[key]]

    training_status = 'Train Step: %d/%d  \n - LOGS - \n'%(current_step, total_training_steps)
    
    for key in logs.keys() :
        training_status += "{} : {:.4f}, ".format(key, logs[key])

    print(training_status)

    if current_step % steps_between_evals:
        for cb in callbacks :
#             print("BATCH_END - {}".format(current_step - 1))
            cb[0].on_batch_end(current_step - 1, {'loss': train_loss})
            
    else :
        print("======MLM TEST======")
        inputs, label = next(train_iterator)
        sub_res = sub_model([inputs['input_ids'], inputs['input_mask'][:, tf.newaxis, tf.newaxis, :], inputs['segment_ids']])
        
        layer_x = model.layers[6]
        layer_x._output_type = 'predicitions'
        output_logits = layer_x([sub_res['sequence_output'], tf.cast(inputs['masked_lm_positions'], dtype = tf.int32)])
        
        prediction = tf.math.exp(output_logits)
        print(tokenizer_for_load.convert_ids_to_tokens(tf.argmax(output_logits[0], axis = 1)))
        print(tokenizer_for_load.convert_ids_to_tokens(inputs['masked_lm_ids'][1]))
        
        lm_labels = inputs['masked_lm_ids'][0]
        lm_output = output_logits[0]
        lm_label_weights = inputs['masked_lm_weights'][0]
        
        masked_lm_accuracy = tf.keras.metrics.sparse_categorical_accuracy(lm_labels, lm_output)
        numerator = tf.reduce_sum(masked_lm_accuracy * lm_label_weights)
        denominator = tf.reduce_sum(lm_label_weights) + 1e-5
        masked_lm_accuracy = numerator / denominator
        
        print("REAL_ACC : {}".format(masked_lm_accuracy))
        
    
    if current_step % steps_per_epoch == 0:
        for cb in callbacks :
#             print("{} EPOCH_END".format(int(current_step / steps_per_epoch)))
            cb[0].on_epoch_end(int(current_step / steps_per_epoch), logs)
            
    print()
for cb in callbacks :
    cb[0].on_train_end()

Train Step: 11/1250  
 - LOGS - 
training_loss : 12.1364, MLM_ACC : 0.4976, MLM_LOSS : 0.5086, NSP_ACC : 0.4981, NSP_LOSS : 0.4991, 

Train Step: 12/1250  
 - LOGS - 
training_loss : 12.0693, MLM_ACC : 0.5036, MLM_LOSS : 0.5147, NSP_ACC : 0.5041, NSP_LOSS : 0.5050, 

Train Step: 13/1250  
 - LOGS - 
training_loss : 12.0233, MLM_ACC : 0.5060, MLM_LOSS : 0.5172, NSP_ACC : 0.5065, NSP_LOSS : 0.5074, 

Train Step: 14/1250  
 - LOGS - 
training_loss : 12.0664, MLM_ACC : 0.5205, MLM_LOSS : 0.5316, NSP_ACC : 0.5208, NSP_LOSS : 0.5219, 

Train Step: 15/1250  
 - LOGS - 
training_loss : 12.0124, MLM_ACC : 0.5223, MLM_LOSS : 0.5334, NSP_ACC : 0.5226, NSP_LOSS : 0.5237, 

Train Step: 16/1250  
 - LOGS - 
training_loss : 11.9828, MLM_ACC : 0.5299, MLM_LOSS : 0.5411, NSP_ACC : 0.5303, NSP_LOSS : 0.5313, 

Train Step: 17/1250  
 - LOGS - 
training_loss : 11.9287, MLM_ACC : 0.5283, MLM_LOSS : 0.5395, NSP_ACC : 0.5287, NSP_LOSS : 0.5296, 

Train Step: 18/1250  
 - LOGS - 
training_loss : 11.8867, MLM_

Train Step: 54/1250  
 - LOGS - 
training_loss : 12.1494, MLM_ACC : 0.5589, MLM_LOSS : 0.5699, NSP_ACC : 0.5594, NSP_LOSS : 0.5604, 

Train Step: 55/1250  
 - LOGS - 
training_loss : 12.2060, MLM_ACC : 0.5615, MLM_LOSS : 0.5725, NSP_ACC : 0.5620, NSP_LOSS : 0.5631, 

Train Step: 56/1250  
 - LOGS - 
training_loss : 12.1740, MLM_ACC : 0.5572, MLM_LOSS : 0.5681, NSP_ACC : 0.5577, NSP_LOSS : 0.5587, 

Train Step: 57/1250  
 - LOGS - 
training_loss : 12.4065, MLM_ACC : 0.5650, MLM_LOSS : 0.5757, NSP_ACC : 0.5654, NSP_LOSS : 0.5667, 

Train Step: 58/1250  
 - LOGS - 
training_loss : 12.3780, MLM_ACC : 0.5612, MLM_LOSS : 0.5719, NSP_ACC : 0.5617, NSP_LOSS : 0.5629, 

Train Step: 59/1250  
 - LOGS - 
training_loss : 12.3454, MLM_ACC : 0.5592, MLM_LOSS : 0.5700, NSP_ACC : 0.5597, NSP_LOSS : 0.5609, 

Train Step: 60/1250  
 - LOGS - 
training_loss : 12.3140, MLM_ACC : 0.5589, MLM_LOSS : 0.5697, NSP_ACC : 0.5594, NSP_LOSS : 0.5606, 
======MLM TEST======
['##븀', '많은데', '중앙에', '개최하는', '방영되었다', '켜'

Train Step: 98/1250  
 - LOGS - 
training_loss : 12.4248, MLM_ACC : 0.5862, MLM_LOSS : 0.5968, NSP_ACC : 0.5868, NSP_LOSS : 0.5881, 

Train Step: 99/1250  
 - LOGS - 
training_loss : 12.4222, MLM_ACC : 0.5882, MLM_LOSS : 0.5988, NSP_ACC : 0.5888, NSP_LOSS : 0.5901, 

Train Step: 100/1250  
 - LOGS - 
training_loss : 12.4114, MLM_ACC : 0.5896, MLM_LOSS : 0.6002, NSP_ACC : 0.5901, NSP_LOSS : 0.5915, 
======MLM TEST======
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']
['썼으며', '##곳', '등이다', '##도', '종류', '.', ',', '##에는', '##지를', '##거나', '##는', '중기', '##톱', '##개', '##망', '돌', ',', '석', '##석기', '##나', '동물을', '##지', '1만', '##날', '##각', '뿔', '##한다', '돌', '쐐', ',', '나뉘', '##원', '##날', '구석기', '있는데', '##기', '##와', '썼다', '[PAD]', '[PAD]']
REAL_ACC : 0.12499985098838806

Train Step: 101/1250  
 - LOGS - 
training_loss : 12.3896, MLM_ACC : 0.5875, 

Train Step: 141/1250  
 - LOGS - 
training_loss : 12.2429, MLM_ACC : 0.5896, MLM_LOSS : 0.6002, NSP_ACC : 0.5902, NSP_LOSS : 0.5916, 

Train Step: 142/1250  
 - LOGS - 
training_loss : 12.2258, MLM_ACC : 0.5898, MLM_LOSS : 0.6003, NSP_ACC : 0.5903, NSP_LOSS : 0.5917, 

Train Step: 143/1250  
 - LOGS - 
training_loss : 12.2247, MLM_ACC : 0.5904, MLM_LOSS : 0.6009, NSP_ACC : 0.5909, NSP_LOSS : 0.5923, 

Train Step: 144/1250  
 - LOGS - 
training_loss : 12.2299, MLM_ACC : 0.5910, MLM_LOSS : 0.6015, NSP_ACC : 0.5915, NSP_LOSS : 0.5929, 

Train Step: 145/1250  
 - LOGS - 
training_loss : 12.2339, MLM_ACC : 0.5900, MLM_LOSS : 0.6006, NSP_ACC : 0.5906, NSP_LOSS : 0.5920, 

Train Step: 146/1250  
 - LOGS - 
training_loss : 12.2167, MLM_ACC : 0.5900, MLM_LOSS : 0.6005, NSP_ACC : 0.5905, NSP_LOSS : 0.5919, 

Train Step: 147/1250  
 - LOGS - 
training_loss : 12.2153, MLM_ACC : 0.5904, MLM_LOSS : 0.6009, NSP_ACC : 0.5909, NSP_LOSS : 0.5923, 

Train Step: 148/1250  
 - LOGS - 
training_loss : 12.20

Train Step: 185/1250  
 - LOGS - 
training_loss : 11.9043, MLM_ACC : 0.5909, MLM_LOSS : 0.6015, NSP_ACC : 0.5915, NSP_LOSS : 0.5928, 

Train Step: 186/1250  
 - LOGS - 
training_loss : 11.8879, MLM_ACC : 0.5908, MLM_LOSS : 0.6013, NSP_ACC : 0.5913, NSP_LOSS : 0.5927, 

Train Step: 187/1250  
 - LOGS - 
training_loss : 11.8931, MLM_ACC : 0.5905, MLM_LOSS : 0.6011, NSP_ACC : 0.5911, NSP_LOSS : 0.5924, 

Train Step: 188/1250  
 - LOGS - 
training_loss : 11.9028, MLM_ACC : 0.5894, MLM_LOSS : 0.6000, NSP_ACC : 0.5900, NSP_LOSS : 0.5913, 

Train Step: 189/1250  
 - LOGS - 
training_loss : 11.8926, MLM_ACC : 0.5907, MLM_LOSS : 0.6012, NSP_ACC : 0.5912, NSP_LOSS : 0.5926, 

Train Step: 190/1250  
 - LOGS - 
training_loss : 11.8918, MLM_ACC : 0.5901, MLM_LOSS : 0.6007, NSP_ACC : 0.5906, NSP_LOSS : 0.5920, 
======MLM TEST======
[',', '.', ',', ',', ',', '.', '.', '.', ',', ',', ',', '.', '.', '.', ',', '.', '.', '.', ',', ',', ',', ',', ',', '.', '.', ',', ',', ',', ',', ',', ',', '.', '.', ',',

Train Step: 230/1250  
 - LOGS - 
training_loss : 11.5449, MLM_ACC : 0.5892, MLM_LOSS : 0.5998, NSP_ACC : 0.5897, NSP_LOSS : 0.5910, 
======MLM TEST======
[',', ',', '.', '.', ',', ',', ',', ',', ',', '.', ',', ',', '.', '.', ',', ',', ',', ',', ',', ',', ',', ',', '.', ',', '.', '.', ',', ',', '.', ',', '.', '.', ',', ',', '.', ',', ',', '.', '.', '.']
['##있는', '들어', '라는', '보면', '유명해졌다', '##원장', '##고', '퍼져', '##만에', '##에', '알려지지', '2', '.', '##이', '9월', ')', '##로', '##들이', '##스', '이틀', '(', '개념이', '##와', '않을', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
REAL_ACC : 0.026315782219171524

Train Step: 231/1250  
 - LOGS - 
training_loss : 11.5387, MLM_ACC : 0.5876, MLM_LOSS : 0.5982, NSP_ACC : 0.5881, NSP_LOSS : 0.5894, 

Train Step: 232/1250  
 - LOGS - 
training_loss : 11.5289, MLM_ACC : 0.5889, MLM_LOSS : 0.5995, NSP_ACC : 0.5894, NSP_LOSS : 0.5907, 

Train Step: 233/1250  
 - LOGS - 
tr

Train Step: 271/1250  
 - LOGS - 
training_loss : 11.2238, MLM_ACC : 0.5905, MLM_LOSS : 0.6012, NSP_ACC : 0.5910, NSP_LOSS : 0.5923, 

Train Step: 272/1250  
 - LOGS - 
training_loss : 11.2194, MLM_ACC : 0.5904, MLM_LOSS : 0.6010, NSP_ACC : 0.5909, NSP_LOSS : 0.5921, 

Train Step: 273/1250  
 - LOGS - 
training_loss : 11.2107, MLM_ACC : 0.5894, MLM_LOSS : 0.6001, NSP_ACC : 0.5899, NSP_LOSS : 0.5912, 

Train Step: 274/1250  
 - LOGS - 
training_loss : 11.2022, MLM_ACC : 0.5905, MLM_LOSS : 0.6011, NSP_ACC : 0.5910, NSP_LOSS : 0.5922, 

Train Step: 275/1250  
 - LOGS - 
training_loss : 11.2008, MLM_ACC : 0.5901, MLM_LOSS : 0.6008, NSP_ACC : 0.5907, NSP_LOSS : 0.5919, 

Train Step: 276/1250  
 - LOGS - 
training_loss : 11.2065, MLM_ACC : 0.5906, MLM_LOSS : 0.6013, NSP_ACC : 0.5912, NSP_LOSS : 0.5924, 

Train Step: 277/1250  
 - LOGS - 
training_loss : 11.2141, MLM_ACC : 0.5904, MLM_LOSS : 0.6011, NSP_ACC : 0.5909, NSP_LOSS : 0.5922, 

Train Step: 278/1250  
 - LOGS - 
training_loss : 11.21

Train Step: 316/1250  
 - LOGS - 
training_loss : 11.0497, MLM_ACC : 0.5926, MLM_LOSS : 0.6032, NSP_ACC : 0.5931, NSP_LOSS : 0.5944, 

Train Step: 317/1250  
 - LOGS - 
training_loss : 11.0409, MLM_ACC : 0.5930, MLM_LOSS : 0.6036, NSP_ACC : 0.5935, NSP_LOSS : 0.5948, 

Train Step: 318/1250  
 - LOGS - 
training_loss : 11.0333, MLM_ACC : 0.5920, MLM_LOSS : 0.6027, NSP_ACC : 0.5925, NSP_LOSS : 0.5938, 

Train Step: 319/1250  
 - LOGS - 
training_loss : 11.0308, MLM_ACC : 0.5925, MLM_LOSS : 0.6031, NSP_ACC : 0.5930, NSP_LOSS : 0.5943, 

Train Step: 320/1250  
 - LOGS - 
training_loss : 11.0266, MLM_ACC : 0.5925, MLM_LOSS : 0.6031, NSP_ACC : 0.5930, NSP_LOSS : 0.5943, 
======MLM TEST======
['.', '.', '.', '.', ',', ',', '.', ',', '.', ',', ',', ',', ',', ',', ',', '.', ',', ',', ',', '.', '.', ',', '.', ',', ',', ',', '.', ',', ',', ',', ',', '.', '.', '.', ',', ',', ',', ',', ',', ',']
['"', '##호인', '"', '전통적인', '진', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.plot(history['training_loss'])
plt.show()

In [ ]:
plt.plot(history['MLM_LOSS'])
plt.show()

In [ ]:
plt.plot(history['NSP_ACC'])
plt.show()

In [ ]:
plt.plot(history['NSP_LOSS'])
plt.show()

In [ ]:
#Test

In [ ]:
inputs, labels = next(iter(dataset))

In [ ]:
sub_res = sub_model([inputs['input_ids'], inputs['input_mask'][:, tf.newaxis, tf.newaxis, :], inputs['segment_ids']])

In [ ]:
layer_x = model.layers[6]
layer_x._output_type = 'predicitions'

In [ ]:
output_logits = layer_x([sub_res['sequence_output'], tf.cast(inputs['masked_lm_positions'], dtype = tf.int32)])

In [ ]:
prediction = tf.math.exp(output_logits)

In [ ]:
tf.argmax(prediction[0], axis = 1)

In [ ]:
inputs['masked_lm_ids'][0]

In [ ]:
from transformers import BertTokenizerFast
tokenizer_for_load = BertTokenizerFast.from_pretrained('./model/BertTokenizer-3000-32000-vocab.txt'
                                                   , strip_accents=False
                                                   , lowercase=False)

In [ ]:
print(tokenizer_for_load.convert_ids_to_tokens(tf.argmax(prediction[1], axis = 1)))

In [ ]:
print(tokenizer_for_load.convert_ids_to_tokens(inputs['masked_lm_ids'][1]))

In [ ]:
lm_labels = inputs['masked_lm_ids'][0]
lm_output = output_logits[0]
lm_label_weights = inputs['masked_lm_weights'][0]

In [ ]:
masked_lm_accuracy = tf.keras.metrics.sparse_categorical_accuracy(lm_labels, lm_output)
numerator = tf.reduce_sum(masked_lm_accuracy * lm_label_weights)
denominator = tf.reduce_sum(lm_label_weights) + 1e-5
masked_lm_accuracy = numerator / denominator

In [ ]:
masked_lm_accuracy